# 0. Table - EDB

## Table 1: Product

### Col 1: item

### Col 2: price

## Table 2: Product

### Col 1: item

### Col 2: country

In [1]:
productCol = ["Product","Price"]
madeinCol = ["Product", "Country"]

In [2]:
tables = ["madein", "price"]

In [3]:
tabCol = {tables[0]:madeinCol, tables[1]:productCol}

In [70]:
countries = ["germany","belgium","uk","china","usa","japan"]
prices = ["5","6","10"]

In [74]:
consDict = {"Country":countries,  "Price": prices}

In [75]:
consDict

{'Country': ['germany', 'belgium', 'uk', 'china', 'usa', 'japan'],
 'Price': ['5', '6', '10']}

# 1. From Datalog to Sql

## Datalog Programs Predicates

## $p = \{p_1,p_2,p_3\}$

### $p_1:$ good_product

- Made in Germany or Belgium

### $p_2:$ bad_product

- Not made in Germany, Belgium or UK

### $p_3:$ perfect_product

- Price < 5 and not a bad product

## Datalog Program P

In [76]:
datalog_goodProd = "good_product(Product):-madein(Product,germany).good_product(Product):-madein(Product,belgium)."

datalog_badProd = "bad_product(Product):-madein(Product,Country), Country!=germany, Country!=belgium, Country!=uk."

datalog_perfProd = "perfect_product(Product):-good_product(Product), not bad_product(belgium), price(Product,Price), Price<5."

## Step 1. Replace Constant with variables

In [77]:
import re

In [78]:
sep1 = '.'
sep2 = ':-'
sep3 = ', '
cmps = ["!=",">","<","<=",">=","=="]

In [79]:
datalog_goodProd
datalog_badProd
datalog_perfProd

'perfect_product(Product):-good_product(Product), not bad_product(belgium), price(Product,Price), Price<5.'

In [80]:
def getClauses(datalog_query):
    return [item for item in datalog_query.split(sep1) if item != '']

In [81]:
def getHeads(datalog_query):
    clauses = getClauses(datalog_query)
    return [clause.split(sep2)[0] for clause in clauses]

In [82]:
def getAllPredicates(datalog_query):
    return [head.split('(')[0] for head in getHeads(datalog_query)]

In [83]:
def getVariablesInHeads(datalog_query):
    heads = getHeads(datalog_query)
    query = []
    for head in heads:
        query += [q.strip() for q in head[head.find("(")+1:head.find(")")].split(',')]
    return query

In [84]:
def getBodies(datalog_query):
    clauses = getClauses(datalog_query)
    return [clause.split(sep2)[1] for clause in clauses]

In [85]:
def getHBDict(datalog_query):
    result = {}
    clauses = getClauses(datalog_query)
    for clause in clauses:
        result[clause.split(sep2)[1]] = clause.split(sep2)[0]+sep2+clause.split(sep2)[1]
    return result

In [86]:
def getTotalPredicates(datalog_query):
    result = []
    for body in getBodies(datalog_query):
        for term in body.split(', '):
            result.append(term)
    for head in getHeads(datalog_query):
        result.append(head)
    return result

In [87]:
def getLiteralHeadDict(datalog_query):
    bH = getHBDict(datalog_query)
    result = {}
    for key in bH.keys():
        if len(key.split(sep3))>1:
            for term in key.split(sep3):
                result[term] = bH[key]
        else:
            result[key.split(sep3)[0]] = bH[key]
    return result

In [88]:
def getVariablesInBodies(datalog_query):
    query = []
    for body in getHBDict(datalog_query).keys():
        for term in body.split(', '):
            if term.find("(") != -1:
                query += [q.strip() for q in term[term.find("(")+1:term.find(")")].split(',')]
            else:
                for cmp in cmps:
                    if term.find(cmp) != -1:
                        query.append(term.split(cmp)[0])
                        query.append(term.split(cmp)[1])
    return query

In [89]:
def getVariablesInClauses(datalog_query):
    query = []
    for body in getHBDict(datalog_query).keys():
        for term in body.split(', '):
            if term.find("(") != -1:
                query += [q.strip() for q in term[term.find("(")+1:term.find(")")].split(',')]
            else:
                for cmp in cmps:
                    if term.find(cmp) != -1:
                        query.append(term.split(cmp)[0])
                        query.append(term.split(cmp)[1])
    for body in getHBDict(datalog_query).values():
        for term in body.split(', '):
            if term.find("(") != -1:
                query += [q.strip() for q in term[term.find("(")+1:term.find(")")].split(',')]
            else:
                for cmp in cmps:
                    if term.find(cmp) != -1:
                        query.append(term.split(cmp)[0])
                        query.append(term.split(cmp)[1])
    return query

In [90]:
def getPVDict(datalog_query):
    predConsDict = {}
    i = 1
    for var in getVariablesInClauses(datalog_query):
        for ls in consDict.values():
            if var in ls:
                if var in predConsDict.keys():
                    continue
                else:
                    predConsDict[var] = 'P'+str(i)
                    i = i + 1
    return predConsDict

In [91]:
pvDict = getPVDict(inp)

In [92]:
inp = datalog_goodProd+datalog_badProd+datalog_perfProd

In [93]:
inp

'good_product(Product):-madein(Product,germany).good_product(Product):-madein(Product,belgium).bad_product(Product):-madein(Product,Country), Country!=germany, Country!=belgium, Country!=uk.perfect_product(Product):-good_product(Product), not bad_product(belgium), price(Product,Price), Price<5.'

In [94]:
bH = getHBDict(inp)

In [95]:
bH

{'good_product(Product), not bad_product(belgium), price(Product,Price), Price<5': 'perfect_product(Product):-good_product(Product), not bad_product(belgium), price(Product,Price), Price<5',
 'madein(Product,Country), Country!=germany, Country!=belgium, Country!=uk': 'bad_product(Product):-madein(Product,Country), Country!=germany, Country!=belgium, Country!=uk',
 'madein(Product,belgium)': 'good_product(Product):-madein(Product,belgium)',
 'madein(Product,germany)': 'good_product(Product):-madein(Product,germany)'}

In [96]:
consDict

{'Country': ['germany', 'belgium', 'uk', 'china', 'usa', 'japan'],
 'Price': ['5', '6', '10']}

In [97]:
pvDict

{'5': 'P4', 'belgium': 'P2', 'germany': 'P1', 'uk': 'P3'}

In [98]:
consts = []
for key in consDict.keys():
    consts+=consDict[key]

In [101]:
newDict = {}
for key in bH.keys():
    i = 1
    clause = bH[key]
    for const in consts:     
        if clause.find(const) != -1:
            print(const)
            variable = "v" + str(i)
            clause = clause.replace(const,variable)
            add = ", " + pvDict[const] + "(" + variable + ")"
            temp = clause + add
            i = i + 1
    newDict[key] = temp 

germany
belgium
uk
germany
5
belgium
belgium


In [102]:
newDict

{'good_product(Product), not bad_product(belgium), price(Product,Price), Price<5': 'perfect_product(Product):-good_product(Product), not bad_product(v2), price(Product,Price), Price<v1, P2(v2)',
 'madein(Product,Country), Country!=germany, Country!=belgium, Country!=uk': 'bad_product(Product):-madein(Product,Country), Country!=v1, Country!=v2, Country!=v3, P3(v3)',
 'madein(Product,belgium)': 'good_product(Product):-madein(Product,v1), P2(v1)',
 'madein(Product,germany)': 'good_product(Product):-madein(Product,v1), P1(v1)'}